# Demo Peg Keeping Logic

One of the agents in our simulation is the `Keeper`. The `Keeper` calls the `update` function on all Peg Keepers if doing so exceeds their profitability threshold (default 0). This notebook demos the `Keeper`'s `update` method.

In [1]:
from src.sim.scenario import Scenario

In [2]:
def increment_timestamps(aggregator, pks):
    ts = aggregator._block_timestamp + 60 * 60  # one hour
    for pk in pks:
        pk._increment_timestamp(ts)
        pk.POOL._increment_timestamp(ts)
        assert pk.last_change != pk._block_timestamp
    aggregator._increment_timestamp(ts)
    print(f"New Aggregator Price: {aggregator.price() / 1e18}")

In [3]:
# Generate markets and prices
scenario = Scenario("baseline")
scenario.prepare_for_run()

[DEBUG][16:44:27][root]-761840: Using 614880 1Inch quotes.
[INFO][16:44:27][root]-761840: Fetching sim_market from subgraph.
[INFO][16:44:30][root]-761840: Found 20 valid cycles of length 3.


In [4]:
aggregator = scenario.aggregator
aggregator.price() / 1e18

0.9953737870756432

In [5]:
pks = scenario.peg_keepers
keeper = scenario.keeper
profit, count = keeper.update(pks)
print(f"Profit: {profit}, Count: {count}")

Profit: 0.0, Count: 0


In all likelihood, no updates occured even though the pools are not balanced. This is because the balance of crvUSD exceeds the balance of the peg stablecoin (e.g. USDC), but the `PegKeeper` doesn't have any debt to withdraw. To test the `update` functionality, we trade against the stableswap pools such that the crvUSD balance is less than the peg stablecoin balance.

In [6]:
for pk in pks:
    spool = pk.POOL
    print(spool.name)

    normalized_balances = [b * r / 1e18 for b, r in zip(spool.balances, spool.rates)]
    print(f"Normalized balances before trade: {normalized_balances}")

    diff = normalized_balances[pk.I] - normalized_balances[pk.I ^ 1]
    diff = int(diff * 1e18 / spool.rates[pk.I ^ 1])  # convert to peg coin units
    print(
        f"Swapping in {diff} {spool.coins[pk.I^1].symbol} for {spool.coins[pk.I].symbol}"
    )

    amt_in, amt_out, fees = spool.trade(pk.I ^ 1, pk.I, diff)
    assert amt_in == diff
    print(f"Received: {amt_out}, Fees: {fees}")
    normalized_balances = [b * r / 1e18 for b, r in zip(spool.balances, spool.rates)]
    print(f"Normalized balances after trade: {normalized_balances}")

    print()

Curve.fi Factory Plain Pool: crvUSD/TUSD
Normalized balances before trade: [1.4000154639228428e+24, 3.203409055350148e+24]
Swapping in 1803393591427305496903680 TUSD for crvUSD
Received: 1803213252068162766449523, Fees: 180339359142730549699
Normalized balances after trade: [3.203409055350148e+24, 1.4001056336024142e+24]

Curve.fi Factory Plain Pool: crvUSD/USDP
Normalized balances before trade: [1.1788968704809905e+24, 3.806845972337115e+24]
Swapping in 2627949101856124221521920 USDP for crvUSD
Received: 2627686306945938610252246, Fees: 262794910185612422267
Normalized balances after trade: [3.806845972337115e+24, 1.1790282679360835e+24]

Curve.fi Factory Plain Pool: crvUSD/USDC
Normalized balances before trade: [7.766160753812e+24, 2.704459766574084e+25]
Swapping in 19278436911928 USDC for crvUSD
Received: 19276509068236811053122338, Fees: 1927843691192800385350
Normalized balances after trade: [2.704459766574e+25, 7.767124675658436e+24]

Curve.fi Factory Plain Pool: crvUSD/USDT
Norm

In [7]:
# Increment timestamps to update aggregator price
# AND pk timestamps
increment_timestamps(aggregator, pks)

New Aggregator Price: 1.0045022563045432


In [8]:
profit, count = keeper.update(pks)
print(f"Profit: {profit / 1e18}, Count: {count}")

[INFO][16:44:30][root]-761840: Updating crvUSD/TUSD Peg Keeper with profit 70.
[INFO][16:44:30][root]-761840: Updating crvUSD/USDP Peg Keeper with profit 257.
[INFO][16:44:30][root]-761840: Updating crvUSD/USDC Peg Keeper with profit 1810.
[INFO][16:44:30][root]-761840: Updating crvUSD/USDT Peg Keeper with profit 3294.


Profit: 3294.3374815760258, Count: 4


In [9]:
for pk in pks:
    spool = pk.POOL
    print(spool.name)

    normalized_balances = [b * r / 1e18 for b, r in zip(spool.balances, spool.rates)]
    print(f"Normalized balances before trade: {normalized_balances}")
    print()

Curve.fi Factory Plain Pool: crvUSD/TUSD
Normalized balances before trade: [3.203402773435072e+24, 1.7607600470550763e+24]

Curve.fi Factory Plain Pool: crvUSD/USDP
Normalized balances before trade: [3.806835917819817e+24, 1.7045817837387366e+24]

Curve.fi Factory Plain Pool: crvUSD/USDC
Normalized balances before trade: [2.7044522594701e+25, 1.1622544446479758e+25]

Curve.fi Factory Plain Pool: crvUSD/USDT
Normalized balances before trade: [2.4582171622217e+25, 1.0422506506983606e+25]



In [10]:
scenario.llamma.metadata["controller_params"]

{'address': '0xa920de414ea4ab66b97da1bfe9e6eca7d4219635',
 'liquidation_discount': '0.06',
 'loan_discount': '0.09',
 'rate': '0.1488105338145901001357639503573224',
 'minted': '1190864825.997993509049775384',
 'redeemed': '1168085785.240306774993871077',
 'future_rate': '0.144414516973859528894954455068666',
 'n_loans': '250'}

In [11]:
# Increment timestamps to update aggregator price
# AND pk timestamps
increment_timestamps(aggregator, pks)

New Aggregator Price: 1.002234942693108


In [12]:
# We can update again! Recall that the PK can only
# deposit/withdraw 20% of the pool's imbalance at a time.
profit, count = keeper.update(pks)
print(f"Profit: {profit / 1e18}, Count: {count}")

[INFO][16:44:30][root]-761840: Updating crvUSD/TUSD Peg Keeper with profit 35.
[INFO][16:44:30][root]-761840: Updating crvUSD/USDP Peg Keeper with profit 113.
[INFO][16:44:30][root]-761840: Updating crvUSD/USDC Peg Keeper with profit 736.
[INFO][16:44:30][root]-761840: Updating crvUSD/USDT Peg Keeper with profit 1322.


Profit: 1321.8637026295173, Count: 4


In [13]:
# Increment timestamps to update aggregator price
# AND pk timestamps
increment_timestamps(aggregator, pks)

New Aggregator Price: 1.0013994123408458
